In [37]:
import neo4j as neo

In [38]:
url = 'bolt://localhost:7687'
auth = ('neo4j', 'neo4j_')

In [55]:
def _run_query(tx, query, **args):
    """
    Run one query statement in transaction, and return result
    """
    result = tx.run(query, **args)
    return result.single()
    
def _run_transaction(statement_list):
    # Make neo4j driver object
    driver = neo.GraphDatabase.driver(url, auth=auth)
    try:
        # Start neo4j session
        with driver.session() as session:
            for statement in statement_list:
                query = statement['query']
                args = statement.get('args', {})
                # Put each statement into transaction
                result = session.write_transaction(_run_query, query, **args)
                print(result)
    finally:
        driver.close()

In [64]:
"""
Create or merge object in neo4j
"""

statements = [
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 1,
            'name': 'Alvin',
            'gender': 'M' 
        }
    },
    {
        'query': 'MERGE (person:Person {id: {id}, name: {name}, gender: {gender}}) '
                 'RETURN person',
        'args': {
            'id': 2,
            'name': 'Emma',
            'gender': 'F' 
        }
    }
]

_run_transaction(statements)

<Record person=<Node id=69984 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>>
<Record person=<Node id=69985 labels={'Person'} properties={'gender': 'F', 'name': 'Emma', 'id': 2}>>


In [70]:
"""
Update exist object
"""

statements = [
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 1,
            'birthday': '1981-03-17'
        }
    },
    {
        'query': 'MATCH (person:Person)'
                 'WHERE person.id = {id} '
                 'SET person.birthday = {birthday} '
                 'RETURN person',
        'args': {
            'id': 2,
            'birthday': '1985-03-29'
        }
    }
]

_run_transaction(statements)

<Record person=<Node id=69984 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}>>
<Record person=<Node id=69985 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}>>


In [77]:
"""
Build relationship
"""

statements = [
    {
        'query': 'MATCH (p1:Person), (p2:Person)'
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p1)-[r:Married]->(p2) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        }
    },
    {
        'query': 'MATCH (p1:Person), (p2:Person) '
                 'WHERE p1.id = {id1} AND p2.id = {id2} '
                 'MERGE (p2)-[r:Married]->(p1) '
                 'RETURN p1, p2, type(r)',
        'args': {
            'id1': 1,
            'id2': 2
        }
    }
]

_run_transaction(statements)

<Record p1=<Node id=69984 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> p2=<Node id=69985 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}> type(r)='Married'>
<Record p1=<Node id=69984 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> p2=<Node id=69985 labels={'Person'} properties={'birthday': '1985-03-29', 'name': 'Emma', 'id': 2, 'gender': 'F'}> type(r)='Married'>


In [79]:
"""
Create object and build relationship
"""

statements = [
    {
        'query': 'MATCH (person:Person) '
                 'WHERE person.id = {person_id} '
                 'MERGE (cat:Cat {name: {cat_name}}) '
                 'MERGE (person)-[r:Feed]->(cat) '
                 'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_name': "Lily"
        }
    }
]

_run_transaction(statements)

<Record person=<Node id=69984 labels={'Person'} properties={'birthday': '1981-03-17', 'name': 'Alvin', 'id': 1, 'gender': 'M'}> cat=<Node id=87082 labels={'Cat'} properties={'name': 'Lily'}> type(r)='Feed'>


In [ ]:
"""
Return 
"""

statements = [
    {
        'query': 'MATCH (person:Person) '
                 'WHERE person.id = {person_id} '
                 'MERGE (cat:Cat {name: {cat_name}}) '
                 'MERGE (person)-[r:Feed]->(cat) '
                 'RETURN person, cat, type(r)',
        'args': {
            'person_id': 1,
            'cat_name': "Lily"
        }
    }
]

_run_transaction(statements)